<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/AdaptDeepSEAvariableInputLength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
import keras

keras.__version__

Using TensorFlow backend.
/users/eprakash/anaconda3/envs/keras_test/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/eprakash/anaconda3/envs/keras_test/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/eprakash/anaconda3/envs/keras_test/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

'1.2.2'

In [3]:
from keras.models import load_model
from keras.models import model_from_json

keras_model_weights = "/users/eprakash/divergent421/model.weights.h5"
keras_model_json = "/users/eprakash/divergent421/model.arch.json"

keras_model = model_from_json(open(keras_model_json).read())
keras_model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
data/genome_data_dir (InputLayer (None, 1000, 4)       0                                            
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 982, 300)      23100       data/genome_data_dir[0][0]       
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 982, 300)      1200        convolution1d_1[0][0]            
____________________________________________________________________________________________________
prelu_1 (PReLU)                  (None, 982, 3

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense,Flatten
from keras.layers.convolutional import Conv1D

new_model = Sequential()
new_model.add(Conv1D(300, 19, input_shape=(400,4)))

for layer in keras_model.layers[2:]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  new_model.add(new_layer)

print(new_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 382, 300)      23100       convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 382, 300)      1200        convolution1d_1[0][0]            
____________________________________________________________________________________________________
prelu_1 (PReLU)                  (None, 382, 300)      114600      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 127, 300)      0           prelu_1[0][0]                    
___________________________________________________________________________________________

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense,Flatten
from keras.layers.convolutional import Conv1D

new_model = Sequential()
new_model.add(Conv1D(300, 19, input_shape=(400,4)))

size_change_indices = [2, 6, 10, 13, 21]

for layer in keras_model.layers[2:22]:
    layer_config = layer.get_config()
    del layer_config['name']
    new_layer = type(layer)(**layer_config)
    new_model.add(new_layer)
    
layer_config = keras_model.layers[22].get_config()
#layer_config['output_dim']= 1
del layer_config['name']
new_layer = type(keras_model.layers[22])(**layer_config)
new_model.add(new_layer)

layer_config = keras_model.layers[23].get_config()
del layer_config['name']
new_layer = type(keras_model.layers[23])(**layer_config)
new_model.add(new_layer)
    
for i in range(len(new_model.layers)):
    if i in size_change_indices:
        continue
    layer = new_model.layers[i]
    if (len(layer.get_weights()) > 0):
        pass
        #layer.build(layer.input_shape)
    layer.set_weights(layer.get_weights())
    
layer = new_model.layers[2]
if (len(layer.get_weights()) > 0):
    pass
    #layer.build(layer.input_shape)
weights = keras_model.layers[3].get_weights()[0][300:682,:]
layer.set_weights([weights])

layer = new_model.layers[6]
if (len(layer.get_weights()) > 0):
    pass
    #layer.build(layer.input_shape)
weights = keras_model.layers[7].get_weights()[0][100:227,:]
layer.set_weights([weights])

layer = new_model.layers[10]
if (len(layer.get_weights()) > 0):
    pass
    #layer.build(layer.input_shape)
weights = keras_model.layers[11].get_weights()[0][24:55,:]
layer.set_weights([weights])

In [7]:
dense_layer = new_model.layers[13]
if (len(dense_layer.get_weights()) > 0):
    pass
    #dense_layer.build(dense_layer.input_shape)

weights, biases = keras_model.layers[14].get_weights()
weights = weights[1300:2700,:]
dense_layer.set_weights([weights, biases])

In [8]:
print(new_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_4 (Convolution1D)  (None, 382, 300)      23100       convolution1d_input_2[0][0]      
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 382, 300)      1200        convolution1d_4[0][0]            
____________________________________________________________________________________________________
prelu_6 (PReLU)                  (None, 382, 300)      114600      batchnormalization_6[0][0]       
____________________________________________________________________________________________________
maxpooling1d_4 (MaxPooling1D)    (None, 127, 300)      0           prelu_6[0][0]                    
___________________________________________________________________________________________

In [ ]:
#new_model.save('divergent421.h5')

In [9]:
adapted_inp = np.random.RandomState(1).rand(20,400,4)

In [10]:
adapted_preds=new_model.predict(adapted_inp)

In [11]:
def pad(seq_len,onehot_seqs,pad_len):
    padded_onehot=np.zeros((onehot_seqs.shape[0],pad_len,onehot_seqs.shape[2]))
    padding=np.zeros((int((pad_len-seq_len)/2),4))
    for i in range(0,onehot_seqs.shape[0]):
        padded_onehot[i]=np.concatenate((padding,onehot_seqs[i],padding))
    return padded_onehot
padded_inp=pad(400, adapted_inp, 1000)

In [13]:
print(adapted_inp.shape)
print(padded_inp.shape)

(20, 400, 4)
(20, 1000, 4)


In [14]:
keras_model.load_weights(keras_model_weights)
orig_preds=keras_model.predict(padded_inp)
print(adapted_preds.shape)
print(orig_preds.shape)

(20, 421)
(20, 421)


In [15]:
np.max(np.abs(adapted_preds-orig_preds))

0.4998657